In [1]:
import numpy as np
import pandas as pd

from datetime import datetime, timedelta

import re

# 1. 定義資料時間

In [2]:
specified_date = datetime(2024, 2, 12)

last_monday = specified_date - timedelta(days = 7)

last_sunday = specified_date - timedelta(days = 1)

print("上周一的日期：", last_monday.strftime("%Y-%m-%d"))
print("上周日的日期：", last_sunday.strftime("%Y-%m-%d"))

上周一的日期： 2024-02-05
上周日的日期： 2024-02-11


# 2. GA4資料

## 2-1 各渠道每日資料

https://lookerstudio.google.com/u/0/reporting/702c2e1a-24cd-4524-8eba-7b828bcd265b/page/3jzoD
#### 表一、每周流量表現/各渠道流量表現

In [3]:
df_GA4 = pd.read_csv("../data/GA4_1_session_source_medium.csv", encoding = "utf-8-sig")

df_GA4_bk1 = df_GA4.copy()

df_GA4.head()

,Date,Session source / medium,Views,Total users,Sessions
0,"Feb 6, 2024",google / organic,371404,278577,319787
1,"Feb 7, 2024",google / organic,366766,278587,331981
2,"Feb 5, 2024",google / organic,345238,254623,298700
3,"Feb 8, 2024",google / organic,328793,256184,298385
4,"Feb 9, 2024",google / organic,309351,239912,272846


### 2-1-1 每日資料 

In [10]:
df_daily_data = df_GA4.groupby(by = "Date").sum().sort_index(ascending = True)

df_daily_data

C:\Users\JoeJian\AppData\Local\Temp\ipykernel_14476\440987.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df_daily_data = df_GA4.groupby(by = "Date").sum()


,Views,Total users,Sessions
Date,,,
"Feb 10, 2024",381752,264523,292972
"Feb 11, 2024",319390,218592,245886
"Feb 5, 2024",763516,501907,571272
"Feb 6, 2024",743542,503757,569600
"Feb 7, 2024",812336,529714,613720
"Feb 8, 2024",628176,439081,499878
"Feb 9, 2024",530149,373539,419825


In [5]:
df_daily_data.to_csv(f"./1.每周流量表現/GA4_daily_data_{str(last_monday.month)}{str(last_monday.day)}_{str(last_sunday.month)}{str(last_sunday.day)}.csv")

### 2-1-2 各渠道加總

In [6]:
df_GA4["Session source / medium"] = df_GA4["Session source / medium"].str.lower()

In [7]:
list_session_source_medium = ["google", "fb|facebook", "line", "direct", "yahoo", "延伸閱讀", "popin", "push", "dable", "tiki", "ig|instagram", "udn"]
list_category_session_source = []
list_views_session_source = []
df_daily_views_session_source = pd.DataFrame()

for i in range(len(list_session_source_medium)):
    Views = df_GA4[df_GA4["Session source / medium"].str.contains(f"{list_session_source_medium[i]}")]["Views"].sum()
    
    print(f"SOURCE: {list_session_source_medium[i]}, Views: {Views}")
        
    list_category_session_source.append(list_session_source_medium[i])
    list_views_session_source.append(Views)
    
    if i <= 3:
        if list_session_source_medium[i] == "fb|facebook":
            file_name = list_session_source_medium[i][:2]
        else:
            file_name = list_session_source_medium[i]    
        df_daily_views_session_source = df_GA4[df_GA4["Session source / medium"].str.contains(f"{list_session_source_medium[i]}")].groupby(by = "Date").sum()
        print(df_daily_views_session_source.head())
        print("\n")
        df_daily_views_session_source.to_csv(f"./1.每周流量表現/GA4_daily_data_{i}_{file_name}.csv")

SOURCE: google, Views: 2155034
               Views  Total users  Sessions
Date                                       
Feb 10, 2024  204354       161095    179651
Feb 11, 2024  164473       128854    148023
Feb 5, 2024   358663       264163    309574
Feb 6, 2024   383014       286683    328762
Feb 7, 2024   382944       290030    344914


SOURCE: fb|facebook, Views: 860291
               Views  Total users  Sessions
Date                                       
Feb 10, 2024   81670        38823     42218
Feb 11, 2024   61139        29988     32111
Feb 5, 2024   162198        76813     83836
Feb 6, 2024   141963        67654     75560
Feb 7, 2024   206333        94744    106177


SOURCE: line, Views: 371889
              Views  Total users  Sessions
Date                                      
Feb 10, 2024  31956        18894     21130
Feb 11, 2024  44482        26138     29366
Feb 5, 2024   69806        38161     41928
Feb 6, 2024   53763        31903     36792
Feb 7, 2024   87475        4

C:\Users\JoeJian\AppData\Local\Temp\ipykernel_14476\3438187176.py:19: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df_daily_views_session_source = df_GA4[df_GA4["Session source / medium"].str.contains(f"{list_session_source_medium[i]}")].groupby(by = "Date").sum()
C:\Users\JoeJian\AppData\Local\Temp\ipykernel_14476\3438187176.py:19: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df_daily_views_session_source = df_GA4[df_GA4["Session source / medium"].str.contains(f"{list_session_source_medium[i]}")].groupby(by = "Date").sum()
C:\Users\JoeJian\AppData\Local\Temp\ipykernel_14476\3438187176.py:19: FutureWarning:

In [8]:
list_push = ["push / text", "app / push"]

for i in range(len(list_push)):
    Views = df_GA4[df_GA4["Session source / medium"].str.contains(f"{list_push[i]}")]["Views"].sum()
    
    print(f"Push: {list_push[i]}, Views: {Views}")
        
    list_category_session_source.append(list_push[i])
    list_views_session_source.append(Views)

list_all_session_source_medium = list_session_source_medium + list_push

Push: push / text, Views: 57252
Push: app / push, Views: 12453


In [9]:
df_views_session_source = pd.DataFrame({"Views": list_views_session_source}, index = list_all_session_source_medium)
df_views_session_source.to_csv("./1.每周流量表現/GA4_total_views_session_source.csv", encoding = "utf-8-sig")

## 2-2 Percent Scrolled 網頁路徑

https://lookerstudio.google.com/u/0/reporting/702c2e1a-24cd-4524-8eba-7b828bcd265b/page/3jzoD
#### 表四、Percent scrolled 網頁路徑

In [11]:
df_GA4_2 = pd.read_csv("../data/GA4_2_percentscrolled_pagepath.csv")

df_GA4_2_bk = df_GA4_2.copy()

df_GA4_2.head()

,Percent scrolled,Page path,Sessions,Total users
0,10,/article/category/183030/post/202401300002/,117580,108996
1,10,/article/category/183027/post/202402060007/,116984,112374
2,10,/article/category/183012/post/202402020030/,110779,110217
3,25,/article/category/183030/post/202401300002/,109903,103112
4,25,/article/category/183027/post/202402060007/,103903,101548


### 2-2-1 去除空值

In [12]:
df_GA4_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4608 entries, 0 to 4607
Data columns (total 4 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Percent scrolled  4608 non-null   int64 
 1   Page path         4608 non-null   object
 2   Sessions          4608 non-null   int64 
 3   Total users       4608 non-null   int64 
dtypes: int64(3), object(1)
memory usage: 144.1+ KB


In [13]:
df_GA4_2[df_GA4_2["Page path"].isna()]

,Percent scrolled,Page path,Sessions,Total users


In [14]:
df_GA4_2.dropna(inplace = True, axis = 0)
df_GA4_2.reset_index(drop = True, inplace = True)

In [15]:
df_GA4_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4608 entries, 0 to 4607
Data columns (total 4 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Percent scrolled  4608 non-null   int64 
 1   Page path         4608 non-null   object
 2   Sessions          4608 non-null   int64 
 3   Total users       4608 non-null   int64 
dtypes: int64(3), object(1)
memory usage: 144.1+ KB


### 2-2-2 找出文章ID

In [16]:
pattern_filter = r"/article/.*/post/(\d+)/"

for i in range(len(df_GA4_2)):
    match = re.match(pattern_filter, df_GA4_2["Page path"][i])
    if match:
        
        # 取得文章ID
        article_id = match.group(1)
        
        # 確保 article_id 是字符串型態 (str)
        article_id = str(article_id)
        print("文章ID:", article_id)

文章ID: 202401300002
文章ID: 202402060007
文章ID: 202402020030
文章ID: 202401300002
文章ID: 202402060007
文章ID: 202402020030
文章ID: 202402070006
文章ID: 202402070006
文章ID: 202402080008
文章ID: 202402080007
文章ID: 202402080008
文章ID: 202402040002
文章ID: 202402040002
文章ID: 202402080007
文章ID: 202402020014
文章ID: 202402020014
文章ID: 202401240056
文章ID: 202402020027
文章ID: 202402080003
文章ID: 202402070007
文章ID: 202402020046
文章ID: 202402020027
文章ID: 202402060013
文章ID: 202402080003
文章ID: 202402070007
文章ID: 202401240056
文章ID: 202402060013
文章ID: 202402020046
文章ID: 202402070033
文章ID: 202402090001
文章ID: 202402060011
文章ID: 202402020034
文章ID: 202402090001
文章ID: 202402050068
文章ID: 202402020034
文章ID: 202402070033
文章ID: 202402020024
文章ID: 202402060009
文章ID: 202402050068
文章ID: 202402070020
文章ID: 202402060009
文章ID: 202402060011
文章ID: 202402020024
文章ID: 202402070020
文章ID: 202401170052
文章ID: 202401170052
文章ID: 202402040012
文章ID: 202401260022
文章ID: 202311220028
文章ID: 202402090002
文章ID: 202402020036
文章ID: 202401260022
文章ID: 202402

In [17]:
# 定義匹配模式的正則表達式
pattern = r"/article/.*/post/(\d+)/"

# 定義一個函數，應用正則表達式並返回文章ID或 "Not Article"
def extract_article_id(url):
    match = re.match(pattern, url)

    if match and match.group(1):
        return np.int(match.group(1))
    
    else:
        return "0"

In [18]:
# 使用 apply 函數創建新的 'article_id' 欄位
df_GA4_2['article_id'] = df_GA4_2['Page path'].apply(extract_article_id)


C:\Users\JoeJian\AppData\Local\Temp\ipykernel_14476\967637069.py:9: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  return np.int(match.group(1))


In [19]:
df_GA4_2.head()

,Percent scrolled,Page path,Sessions,Total users,article_id
0,10,/article/category/183030/post/202401300002/,117580,108996,202401300002
1,10,/article/category/183027/post/202402060007/,116984,112374,202402060007
2,10,/article/category/183012/post/202402020030/,110779,110217,202402020030
3,25,/article/category/183030/post/202401300002/,109903,103112,202401300002
4,25,/article/category/183027/post/202402060007/,103903,101548,202402060007


In [20]:
not_article_id = []

for index, percent_scrolled, page_path, sessions, users, article_id in df_GA4_2.itertuples():
    # (index, Page path, Views, article_id)
    if article_id == "0":
        not_article_id.append(index)

In [21]:
print(len(not_article_id))
not_article_id[:10]

236


[108, 110, 128, 175, 209, 225, 230, 287, 313, 326]

In [22]:
print("調整前資料筆數:", len(df_GA4_2))

df_GA4_2.drop(not_article_id, inplace = True)

print("調整後資料筆數:", len(df_GA4_2))

調整前資料筆數: 4608
調整後資料筆數: 4372


### 2-2-3 10, 25, 75, 90 工作階段前10名文章

In [23]:
list_percent_scrolled = [10, 25, 75, 90]

In [24]:
df_percent_scrolled_top10_all = pd.DataFrame()

for i in range(len(list_percent_scrolled)):
    
    df_percent_scrolled_top10 = df_GA4_2[df_GA4_2["Percent scrolled"] == list_percent_scrolled[i]].sort_values(by = "Sessions", ascending = False).head(10)
    df_percent_scrolled_top10_all = pd.concat([df_percent_scrolled_top10_all, df_percent_scrolled_top10], axis = 0, ignore_index = True)

In [25]:
df_percent_scrolled_top10_all

,Percent scrolled,Page path,Sessions,Total users,article_id
0,10,/article/category/183030/post/202401300002/,117580,108996,202401300002
1,10,/article/category/183027/post/202402060007/,116984,112374,202402060007
2,10,/article/category/183012/post/202402020030/,110779,110217,202402020030
3,10,/article/category/183016/post/202402070006/,88962,89370,202402070006
4,10,/article/category/183012/post/202402080008/,84583,83258,202402080008
5,10,/article/category/183030/post/202402080007/,83465,77893,202402080007
6,10,/article/category/183015/post/202402040002/,79375,75521,202402040002
7,10,/article/category/183022/post/202402020014/,40331,40097,202402020014
8,10,/article/category/183035/post/202401240056/,32430,32067,202401240056
9,10,/article/category/183016/post/202402020027/,31409,30368,202402020027


In [26]:
df_percent_scrolled_top10_all.to_csv("./2.Scrolling(網頁捲動比例)/GA4_percent_scrolled_pagepath.csv", index = False)

## 2-3 網頁路徑及渠道(Top10)

In [27]:
## 後台資料
df_article_data = pd.read_excel("../data/Ga4報表_今周刊_25000_20240205_0211.xlsx")

df_article_data.head()

,文章ID,文章標題,瀏覽量,文章來源,文章屬性,名人專區,撰文,今周文章母分類,今周文章子分類,ESG文章母分類,ESG文章子分類,熟齡文章母分類,熟齡文章子分類,SEO分類,文章字數,上稿者,最後修改者,文章發布
0,202401300002,哪裡可以換新鈔？Google換鈔地圖輕鬆查！ATM、郵局「限時5天」...父母、小孩紅包行情一次看,229819,數位部記者,生活消費,NaN,黃明惠 整理,職場生活,生活消費,NaN,NaN,NaN,NaN,新鈔/銀行新鈔/郵局新鈔/換新鈔地點/換新鈔2024/新鈔ATM/新鈔提款機/Google換...,1891,黃明惠,李易紓,2024-02-07
1,202402020030,銀行存款超過200萬的人有什麼共通點？500位20-49歲儲蓄調查4結果：存得了錢的人都著眼在未來,228243,外稿/書摘/廣編(無稿費可轉載),投資理財,精選書摘,KURAMA,投資理財,理財,NaN,NaN,NaN,NaN,存錢/存款/儲蓄/雙薪家庭/家庭主婦,1181,歐陽蓉,歐陽蓉,2024-02-05
2,202402070006,ONE BOY傳出財務問題，撤大型廣告、積欠供應商貨款？33歲創辦人顏瑋廷出面回應了,227121,編輯部記者,產業動態,NaN,徐采薇,產業時事,傳產,NaN,NaN,NaN,NaN,ONE BOY/ONE BOY倒閉/ONE BOY財務問題/ONE BOY代言人/ONE B...,983,黃明惠,李易紓,2024-02-07
3,202402060007,2024春節疏運必看：國道省道怎麼開不塞車？何時暫停收費？哪幾天高乘載？一次搞懂返鄉路,198978,數位部記者,生活消費,NaN,胡肇芳 整理,產業時事,政治社會,NaN,NaN,NaN,NaN,過年高乘載/過年交通管制2024/國道/高乘載/塞車路段/過年高鐵/匝道管制,1860,肇芳,肇芳,2024-02-12
4,202402040002,繼黃仁勳、蘇姿丰後又一AI巨星誕生！同樣出身美超微的台灣囡仔 謝金河：出身嘉義的他最感謝這位...,190649,外稿/書摘/廣編(無稿費可轉載),產業動態,老謝開講,謝金河,產業時事,科技,ESG基金與投資,ESG投資趨勢,NaN,NaN,輝達/AI/黃仁勳/蘇姿丯/超微,1092,林依榕,洪慧如,2024-02-04


In [28]:
## 前10名文章ID抓出來
list_top10_article_id = list(df_article_data["文章ID"][:10])

In [29]:
## GA4資料
df_GA4_3 = pd.read_csv("../data/GA4_3_top10_pagepath.csv")

df_GA4_3.head()

,Page path,Session source / medium,Views,Average session duration,Engagement rate
0,/article/category/183030/post/202401300002/,google / organic,207571,40.270353,0.802643
1,/article/category/183027/post/202402060007/,google / organic,180511,23.538525,0.786695
2,/article/category/183012/post/202402020030/,google / organic,122892,23.059344,0.824753
3,/article/category/183016/post/202402070006/,FB / article,115879,17.493689,0.933759
4,/article/category/183030/post/202402080007/,google / organic,113491,32.082765,0.759618


### 2-3-1 去除page path為空值資料

In [30]:
df_GA4_3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 111362 entries, 0 to 111361
Data columns (total 5 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   Page path                 111359 non-null  object 
 1   Session source / medium   111362 non-null  object 
 2   Views                     111362 non-null  int64  
 3   Average session duration  111362 non-null  float64
 4   Engagement rate           111362 non-null  float64
dtypes: float64(2), int64(1), object(2)
memory usage: 4.2+ MB


In [31]:
df_GA4_3[df_GA4_3["Page path"].isna()]

,Page path,Session source / medium,Views,Average session duration,Engagement rate
2661,NaN,google / cpc,91,75.501746,0.907407
44825,NaN,google / organic,2,561.488284,1.000000
80924,NaN,udn.com / referral,1,0.000000,1.000000


In [32]:
df_GA4_3.dropna(inplace = True, axis = 0)
df_GA4_3.reset_index(inplace = True, drop = True)

df_GA4_3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 111359 entries, 0 to 111358
Data columns (total 5 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   Page path                 111359 non-null  object 
 1   Session source / medium   111359 non-null  object 
 2   Views                     111359 non-null  int64  
 3   Average session duration  111359 non-null  float64
 4   Engagement rate           111359 non-null  float64
dtypes: float64(2), int64(1), object(2)
memory usage: 4.2+ MB


### 2-3-2 找出文章ID

In [33]:
pattern_filter = r"/article/.*/post/(\d+)/"

for i in range(len(df_GA4_3)):

    match = re.match(pattern_filter, df_GA4_3["Page path"][i])
    if match:
        
        # 取得文章ID
        article_id = match.group(1)
        
        # 確保 article_id 是字符串型態 (str)
        article_id = str(article_id)
        print("文章ID:", article_id)

文章ID: 202401300002
文章ID: 202402060007
文章ID: 202402020030
文章ID: 202402070006
文章ID: 202402080007
文章ID: 202402040002
文章ID: 202402080008
文章ID: 202402070006
文章ID: 202402020030
文章ID: 202402040002
文章ID: 202402020014
文章ID: 202402020046
文章ID: 202401240056
文章ID: 202402080008
文章ID: 202212120017
文章ID: 202402070007
文章ID: 202402060013
文章ID: 202402040002
文章ID: 202402080008
文章ID: 202402070006
文章ID: 202402070033
文章ID: 202402080003
文章ID: 202402020027
文章ID: 202402020014
文章ID: 202305250031
文章ID: 202401170052
文章ID: 202402020027
文章ID: 202402050068
文章ID: 202402050034
文章ID: 202402020030
文章ID: 202205130037
文章ID: 202204190011
文章ID: 202402020027
文章ID: 202401240056
文章ID: 202402090004
文章ID: 202402020034
文章ID: 202402020024
文章ID: 202402060011
文章ID: 202402080003
文章ID: 202308300001
文章ID: 202402090001
文章ID: 202402050031
文章ID: 202402060009
文章ID: 202402070020
文章ID: 202402020036
文章ID: 202402060042
文章ID: 202311220028
文章ID: 202402070007
文章ID: 202402080009
文章ID: 202402060013
文章ID: 201807020038
文章ID: 202402020046
文章ID: 202402

文章ID: 202402040020
文章ID: 202205130053
文章ID: 202308070032
文章ID: 202310130004
文章ID: 202209250018
文章ID: 202304060024
文章ID: 202311260006
文章ID: 201805180045
文章ID: 202401100017
文章ID: 202311050009
文章ID: 201910160025
文章ID: 201910040027
文章ID: 201912110004
文章ID: 202301160012
文章ID: 202312180031
文章ID: 202005110001
文章ID: 202201270033
文章ID: 202402050018
文章ID: 202402040005
文章ID: 202402020030
文章ID: 201604060024
文章ID: 200311130007
文章ID: 202207170017
文章ID: 202401260022
文章ID: 200001200044
文章ID: 202001060026
文章ID: 202001100027
文章ID: 202302080029
文章ID: 202305220036
文章ID: 202203240031
文章ID: 201310240019
文章ID: 201903070016
文章ID: 201711290019
文章ID: 202401180019
文章ID: 202007230019
文章ID: 202103040030
文章ID: 202206200027
文章ID: 200404150014
文章ID: 202210170063
文章ID: 202402080007
文章ID: 202305050015
文章ID: 202309080019
文章ID: 201504260002
文章ID: 202104260011
文章ID: 202310250046
文章ID: 201901210033
文章ID: 202402030002
文章ID: 202401310014
文章ID: 202103170061
文章ID: 202311160026
文章ID: 202401260022
文章ID: 202212070004
文章ID: 202401

文章ID: 202402070009
文章ID: 202305250049
文章ID: 201907190022
文章ID: 202401240006
文章ID: 201601210019
文章ID: 202304060009
文章ID: 202402040002
文章ID: 202402050012
文章ID: 202103150008
文章ID: 201203220036
文章ID: 202011160003
文章ID: 202205260016
文章ID: 202402070021
文章ID: 202208030034
文章ID: 202002200029
文章ID: 202306200014
文章ID: 202203070003
文章ID: 202402010008
文章ID: 202401020044
文章ID: 200009240006
文章ID: 202306210029
文章ID: 201806010013
文章ID: 202010140032
文章ID: 202302200016
文章ID: 202402070033
文章ID: 202304110039
文章ID: 202203140006
文章ID: 201004010035
文章ID: 200208220051
文章ID: 201511130025
文章ID: 202005100010
文章ID: 200907230002
文章ID: 201202230003
文章ID: 202401250020
文章ID: 200903260018
文章ID: 202112290068
文章ID: 201302280012
文章ID: 202306280009
文章ID: 201910030022
文章ID: 202402070008
文章ID: 201910180003
文章ID: 202204250049
文章ID: 202309130040
文章ID: 202402070033
文章ID: 202401170023
文章ID: 202309040020
文章ID: 201806050020
文章ID: 202402040013
文章ID: 202102220003
文章ID: 202402050019
文章ID: 202305290043
文章ID: 202402070002
文章ID: 202307

文章ID: 201707060020
文章ID: 202105310053
文章ID: 202402010008
文章ID: 202105270009
文章ID: 202401150009
文章ID: 202102050009
文章ID: 202310200052
文章ID: 200004270005
文章ID: 202306140067
文章ID: 202104140015
文章ID: 202104280028
文章ID: 202009180020
文章ID: 202308310005
文章ID: 200005250005
文章ID: 202302080016
文章ID: 202401270002
文章ID: 201501120003
文章ID: 202310180082
文章ID: 202402090004
文章ID: 201708280029
文章ID: 201909230036
文章ID: 202307260053
文章ID: 202104010017
文章ID: 202402020022
文章ID: 201808130016
文章ID: 202401310044
文章ID: 202008240020
文章ID: 202401240006
文章ID: 202402020035
文章ID: 201409290010
文章ID: 202305200001
文章ID: 202401080039
文章ID: 200205160006
文章ID: 201608100006
文章ID: 201602030015
文章ID: 202306190079
文章ID: 199912230011
文章ID: 202312130049
文章ID: 202301100019
文章ID: 202402110002
文章ID: 199912230017
文章ID: 201801100011
文章ID: 200710180021
文章ID: 202002130004
文章ID: 200205230039
文章ID: 200106140035
文章ID: 202401030045
文章ID: 201912190027
文章ID: 202312190005
文章ID: 201604270022
文章ID: 202312280016
文章ID: 201706020012
文章ID: 202106

文章ID: 202303080014
文章ID: 202303100031
文章ID: 202303270026
文章ID: 202303270026
文章ID: 202303290064
文章ID: 202303300018
文章ID: 202304100035
文章ID: 202304190010
文章ID: 202304280037
文章ID: 202305030055
文章ID: 202305110004
文章ID: 202305120038
文章ID: 202305120039
文章ID: 202305120042
文章ID: 202305160023
文章ID: 202305220003
文章ID: 202305250014
文章ID: 202306110021
文章ID: 202306120004
文章ID: 202306120018
文章ID: 202306130045
文章ID: 202306130049
文章ID: 202306140003
文章ID: 202306140028
文章ID: 202306190075
文章ID: 202306190075
文章ID: 202306200007
文章ID: 202307070013
文章ID: 202307280041
文章ID: 202307280041
文章ID: 202308080018
文章ID: 202308080054
文章ID: 202308130014
文章ID: 202308160065
文章ID: 202308230004
文章ID: 202308300061
文章ID: 202309060010
文章ID: 202309080006
文章ID: 202309140024
文章ID: 202309180044
文章ID: 202309200083
文章ID: 202309220027
文章ID: 202309270046
文章ID: 202309280020
文章ID: 202310030014
文章ID: 202310030014
文章ID: 202310160007
文章ID: 202310160047
文章ID: 202310230045
文章ID: 202310250067
文章ID: 202310270018
文章ID: 202310270018
文章ID: 202310

文章ID: 202312300003
文章ID: 202402060009
文章ID: 201710210037
文章ID: 202310160019
文章ID: 202402010032
文章ID: 202010120027
文章ID: 202203300012
文章ID: 202401180032
文章ID: 202309250019
文章ID: 202311030013
文章ID: 202303100031
文章ID: 202402070007
文章ID: 202401150025
文章ID: 202310100003
文章ID: 202301120021
文章ID: 202106130018
文章ID: 202303020028
文章ID: 202007300001
文章ID: 202302130020
文章ID: 202001160017
文章ID: 201612150013
文章ID: 202401250025
文章ID: 202401210001
文章ID: 202302040029
文章ID: 202401080001
文章ID: 201911270052
文章ID: 202306070027
文章ID: 200912240024
文章ID: 201807180015
文章ID: 202302120015
文章ID: 202211300069
文章ID: 202311060011
文章ID: 201903260007
文章ID: 202401180036
文章ID: 202211050005
文章ID: 202211090046
文章ID: 202312310007
文章ID: 202310110076
文章ID: 202310300007
文章ID: 202203040019
文章ID: 202401290037
文章ID: 202305070014
文章ID: 202308180012
文章ID: 202212230029
文章ID: 202212260025
文章ID: 202312020001
文章ID: 201810240020
文章ID: 202401140020
文章ID: 202308090055
文章ID: 202211120004
文章ID: 202301030046
文章ID: 202312010031
文章ID: 202209

文章ID: 201906020006
文章ID: 201906020007
文章ID: 201906030019
文章ID: 201906030031
文章ID: 201906040035
文章ID: 201906080004
文章ID: 201906080004
文章ID: 201906090005
文章ID: 201906110003
文章ID: 201906130044
文章ID: 201906140034
文章ID: 201906170016
文章ID: 201906170024
文章ID: 201906190045
文章ID: 201906210009
文章ID: 201906230008
文章ID: 201906240016
文章ID: 201907040026
文章ID: 201907060004
文章ID: 201907150006
文章ID: 201907150019
文章ID: 201907160035
文章ID: 201907170055
文章ID: 201907190007
文章ID: 201907190017
文章ID: 201907210001
文章ID: 201907240054
文章ID: 201908050029
文章ID: 201707270015
文章ID: 202208180027
文章ID: 202208180040
文章ID: 202208190021
文章ID: 202208310038
文章ID: 202309130034
文章ID: 202309130034
文章ID: 202309130034
文章ID: 202309150025
文章ID: 202003310045
文章ID: 202003310045
文章ID: 201911050033
文章ID: 201911050034
文章ID: 201911180035
文章ID: 201311210012
文章ID: 201512160015
文章ID: 201603290006
文章ID: 201710200037
文章ID: 201808270041
文章ID: 202005060071
文章ID: 201512300003
文章ID: 201805090014
文章ID: 201807040058
文章ID: 201811220011
文章ID: 201812

文章ID: 202210220010
文章ID: 202210220010
文章ID: 202210230012
文章ID: 202210260066
文章ID: 202210260066
文章ID: 202211100019
文章ID: 202309200065
文章ID: 202309200065
文章ID: 202309260036
文章ID: 202310200019
文章ID: 202209070066
文章ID: 202211160009
文章ID: 202211160057
文章ID: 202211230045
文章ID: 202211300013
文章ID: 202212030002
文章ID: 202212030002
文章ID: 202212070022
文章ID: 202212070022
文章ID: 202212090012
文章ID: 202212090012
文章ID: 202212130025
文章ID: 202212130025
文章ID: 202212190013
文章ID: 202210120036
文章ID: 202211070042
文章ID: 202303090032
文章ID: 202305030061
文章ID: 202305030065
文章ID: 202305030065
文章ID: 202305040018
文章ID: 202305040020
文章ID: 202306020013
文章ID: 202306020016
文章ID: 202308010051
文章ID: 202308010054
文章ID: 202310040020
文章ID: 202310040030
文章ID: 202311300038
文章ID: 202311300042
文章ID: 202312040042
文章ID: 202301180036
文章ID: 202302010057
文章ID: 202302010060
文章ID: 202302030018
文章ID: 202307030005
文章ID: 202307030007
文章ID: 202309250030
文章ID: 202309270059
文章ID: 202312060075
文章ID: 202312200007
文章ID: 202310120028
文章ID: 202310

In [34]:
# 使用 apply 函數創建新的 'article_id' 欄位
df_GA4_3['article_id'] = df_GA4_3['Page path'].apply(extract_article_id)


C:\Users\JoeJian\AppData\Local\Temp\ipykernel_14476\967637069.py:9: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  return np.int(match.group(1))


In [35]:
df_GA4_3.head()

,Page path,Session source / medium,Views,Average session duration,Engagement rate,article_id
0,/article/category/183030/post/202401300002/,google / organic,207571,40.270353,0.802643,202401300002
1,/article/category/183027/post/202402060007/,google / organic,180511,23.538525,0.786695,202402060007
2,/article/category/183012/post/202402020030/,google / organic,122892,23.059344,0.824753,202402020030
3,/article/category/183016/post/202402070006/,FB / article,115879,17.493689,0.933759,202402070006
4,/article/category/183030/post/202402080007/,google / organic,113491,32.082765,0.759618,202402080007


In [36]:
not_article_id = []

for index, page_path, source_medium, views, duration, rate, article_id in df_GA4_3.itertuples():
    # (index, Page path, Views, article_id)
    if article_id == "0":
        not_article_id.append(index)

In [37]:
print(len(not_article_id))
not_article_id[:10]

8701


[74, 82, 149, 171, 196, 214, 301, 434, 473, 581]

In [38]:
print("調整前資料筆數:", len(df_GA4_3))

df_GA4_3.drop(not_article_id, inplace = True)

print("調整後資料筆數:", len(df_GA4_3))

調整前資料筆數: 111359
調整後資料筆數: 102658


### 2-3-3 整理Top10資料，包含來源媒介前三名、停留時間與參與度

In [39]:
df_top10_data = pd.DataFrame()
list_session_duration = []
list_engagement_rate = []

for i in range(len(list_top10_article_id)):

    ## -- 1. 前三大流量來源 -- ##
    df_top10_data_3 = df_GA4_3[df_GA4_3["article_id"] == list_top10_article_id[i]].sort_values(by = "Views", ascending = False).head(3)[["article_id", "Session source / medium", "Views"]]

    df_top10_data = pd.concat([df_top10_data, df_top10_data_3], axis = 0, ignore_index = True)
    
    df_top10_data.to_csv("./3.百大文章細分及各渠道來源/GA4_top10_views_3_source_medium.csv", index = False)
    
    
    ## -- 2. 停留時間及參與度
    avg_session_duration = df_GA4_3[df_GA4_3["article_id"] == list_top10_article_id[i]]["Average session duration"]
    avg_engagement_rate = df_GA4_3[df_GA4_3["article_id"] == list_top10_article_id[i]]["Engagement rate"]
    article_views = df_GA4_3[df_GA4_3["article_id"] == list_top10_article_id[i]]["Views"]
    
    
    list_session_duration.append(sum(avg_session_duration * article_views) / sum(article_views))
    list_engagement_rate.append(sum(avg_engagement_rate * article_views) / sum(article_views))


In [40]:
df_top10_other_data = pd.DataFrame({"avg_session_duration": list_session_duration,
                                    "avg_engagement_rate": list_engagement_rate},
                                   index = list_top10_article_id)
df_top10_other_data.to_csv("./3.百大文章細分及各渠道來源/GA4_top10_other_data.csv", index = False)

In [52]:
# list_session_source_medium = ["google", "fb|facebook", "line", "direct", "yahoo", "延伸閱讀", "popin", "push", "dable", "tiki", "ig|instagram", "udn"]
list_category_session_source = []
list_views_session_source = []

df_top10_article = df_GA4_3[df_GA4_3["article_id"].isin(list_top10_article_id)]
df_top10_article["Session source / medium"] = df_top10_article["Session source / medium"].str.lower()

for i in range(4):
    Views = df_top10_article[df_top10_article["Session source / medium"].str.contains(f"{list_session_source_medium[i]}")]["Views"].sum()
    
    print(f"SOURCE: {list_session_source_medium[i]}, Views: {Views}")
        
    list_category_session_source.append(list_session_source_medium[i])
    list_views_session_source.append(Views)

df_top10_4_sources_views = pd.DataFrame({"Views": list_views_session_source}, index = list_category_session_source).sort_values(by = "Views", ascending = False)
df_top10_4_sources_views.to_csv("./3.百大文章細分及各渠道來源/GA4_top10_4_sources_views.csv")

SOURCE: google, Views: 904977
SOURCE: fb|facebook, Views: 311870
SOURCE: line, Views: 66142
SOURCE: direct, Views: 203299


C:\Users\JoeJian\AppData\Local\Temp\ipykernel_14476\3194636895.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_top10_article["Session source / medium"] = df_top10_article["Session source / medium"].str.lower()
